In [ ]:
! pip install -r requirements.txt --quiet

# Semantic Kernel Sequential Orchestration

In sequential orchestration, agents are organized in a pipeline. Each agent processes the task in turn, passing its output to the next agent in the sequence. 

🔗 [Sequential Orchestration](https://learn.microsoft.com/en-us/semantic-kernel/frameworks/agent/agent-orchestration/sequential?pivots=programming-language-python)


<img src="../../media/multi-agent-sequential.png" alt="Diagram" style="height:450px; width:auto;">


In [49]:
from semantic_kernel.agents import SequentialOrchestration
from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent,ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from dotenv import load_dotenv
from os import environ
import asyncio
from semantic_kernel.connectors.mcp import MCPStreamableHttpPlugin
from tracing import set_up_all

load_dotenv(override=True)

kernel = Kernel()


In [50]:
set_up_all(connection_string=environ.get("AZURE_INSIGHT_CONNECTION_STRING"))

In [51]:
chatCompletion = AzureChatCompletion(
    service_id="chat",
    deployment_name=environ["AZURE_OPENAI_MODEL"],
    endpoint=environ["AZURE_OPENAI_ENDPOINT"],
    api_key=environ["AZURE_OPENAI_API_KEY"],
     api_version=environ["AZURE_OPENAI_API_VERSION"] )


reasoningCompletion = AzureChatCompletion(
    service_id="reasoning",
    deployment_name='o3-mini',
    endpoint=environ["AZURE_OPENAI_ENDPOINT"],
    api_key=environ["AZURE_OPENAI_API_KEY"],
     api_version=environ["AZURE_OPENAI_API_VERSION"] )




In [52]:
sales_plugin = MCPStreamableHttpPlugin(
    name="sales",
    url=f"{environ['MCP_SERVER_URL']}",
)

await sales_plugin.connect()


In [53]:

# Sales Data Agent
sales_data_agent = ChatCompletionAgent(
    name="SalesDataAgent",
    instructions=(
        """You are a Sales Data Agent. Your job is to:
        - Dynamically choose and call the appropriate tool(s) to answer user questions about sales, customers, or products.
        - Always include customer names alongside IDs in any output.
        - Always display monetary amounts in $USD (e.g., $123.45).
        - Use filters and aggregations as needed to generate insights from sales orders, products, and customers.
        - Compute totals, revenue, discounts, and other metrics from nested order data.
        - Return **complete, detailed outputs** without summarization, so that all data is available for downstream analysis.
        - Preserve all intermediate results and raw values in your response.
        - Avoid hardcoding analytics; rely on the MCP tools and their parameters.
        - Clarify ambiguous queries before performing analysis.
        - Do not ask the user follow-up questions; only return the requested records.
        - Treat the tools as the source of truth; do not expose raw database internals.
        - return output in table format when possible for consumption by the next agent.
        """
    ),
    plugins=[sales_plugin],
    service=chatCompletion,
)

# Data Analysis Agent
data_analysis_agent = ChatCompletionAgent(
    name="DataAnalysisAgent",
    instructions=(
        """You are a Data Analysis Agent. Given the sales data, your job is to:
        - Analyze and interpret the sales data provided by the SalesDataAgent.
        - Identify trends, patterns, and insights from the sales data.
        - Provide actionable recommendations based on the analysis.
        - Summarize key findings in a clear and concise manner.
        - Ensure that all analyses are relevant to the user's query and objectives.
        - Provide Key Findings, Insights, Trends, Recommendations and a Summary in your final output.
        """
    ),
      service=reasoningCompletion,
)

### Optional: Observe Agent Responses

In [54]:
from semantic_kernel.contents import ChatMessageContent

def agent_response_callback(message: ChatMessageContent) -> None:
    print(f"# {message.name}\n{message.content}")

### Set Up the Sequential Orchestration

In [55]:
agents = [sales_data_agent, data_analysis_agent]
sequential_orchestration = SequentialOrchestration(
    members=agents,
    agent_response_callback=agent_response_callback,
)

### Start the Runtime

In [56]:
from semantic_kernel.agents.runtime import InProcessRuntime

runtime = InProcessRuntime()
runtime.start()

# SalesDataAgent

# SalesDataAgent

# SalesDataAgent
Based on the orders retrieved, here are insights into purchasing behavior and actionable strategies to increase sales for the top 5 customers by number of orders:

### Top 5 Customers by Number of Orders:
| Customer ID | Customer Name | Total Orders | Total Revenue ($USD) |
|-------------|---------------|--------------|-----------------------|
| 3           | Bosch         | 35           | 73,925.38            |
| 4           | NAPA          | 30           | 37,812.34            |
| 0           | Ford          | 25           | 30,512.5             |
| 2           | AutoZone      | 22           | 28,645.29            |
| 1           | GM            | 21           | 27,968.21            |

### Insights into Purchasing Behavior:
1. **Bosch**
   - **Main Products Purchased:** Brake Pads, Oil Filters, and Spark Plugs.
   - **Revenue Contribution:** Highest overall revenue.
   - **Regions:** Orders span NA and EU regions.
   - **Quantity P

In [57]:
orchestration_result = await sequential_orchestration.invoke(
    task="Find the top 5 customers by number of orders, and bring back order and analyze their purchasing behavior to provide actionable insights for increasing sales.",
    runtime=runtime,
)

In [58]:
value = await orchestration_result.get(timeout=30)
print(f"***** Final Result *****\n{value}")

TimeoutError: 

In [ ]:
await runtime.stop_when_idle()